# DrakeVisualizer.jl
This notebook demonstrates interacting with the Drake Visualizer app from Julia. On Mac and Ubuntu, just installing `DrakeVisualizer.jl` will also install a pre-built version of the standalone `drake-visualizer` executable. 

If you're not on one of those platforms, you'll need to install Director (which includes `drake-visualizer`) from <https://github.com/robotlocomotion/director>

In [1]:
# Activate the DrakeVisualizer package, and import some other 
# useful functions
using DrakeVisualizer
using CoordinateTransformations
using Interact
import GeometryTypes: HyperRectangle, Vec, HomogenousMesh
import ColorTypes: RGBA

In [2]:
# Launch the viewer application if it isn't running already:
DrakeVisualizer.any_open_windows() || DrakeVisualizer.new_window();

In [3]:
# First, we'll create a simple geometric object
box = HyperRectangle(Vec(0.,0,0), Vec(1.,1,1))

GeometryTypes.HyperRectangle{3,Float64}(Vec(0.0,0.0,0.0),Vec(1.0,1.0,1.0))

In [4]:
# Visualizer() causes the viewer to spawn a geometry or a set of geometries. 
# It returns a Visualizer, which includes all the information
# about that loaded geometry. 
# Note that the model is initially loaded in the zero configuration 
# (that is, its position and rotation are all zeros)
model = Visualizer(box)

Visualizer with path prefix Symbol[] using LCM LCMCore.LCM(Ptr{Void} @0x00007ff953428200,RawFD(47),LCMCore.Subscription[LCMCore.Subscription{LCMCore.SubscriptionOptions{DrakeVisualizer.Comms.CommsT,DrakeVisualizer.#handle_msg#7{DrakeVisualizer.CoreVisualizer}}}(LCMCore.SubscriptionOptions{DrakeVisualizer.Comms.CommsT,DrakeVisualizer.#handle_msg#7{DrakeVisualizer.CoreVisualizer}}(DrakeVisualizer.Comms.CommsT,DrakeVisualizer.handle_msg),Ptr{Void} @0x00007ff957700ae0)])

In [5]:
# We can use settransform!() to tell the viewer to draw the box at a specific
# position. Translation() creates a CoordinateTransformations.Transformation
# corresponding to the given x; y; z translation.
settransform!(model, Translation(1.,0,0))

true

In [6]:
# We can also rotate the model by sending a different transformation
settransform!(model, LinearMap(AngleAxis(pi/4, 0, 0, 1)))

true

In [7]:
# And we can clear the box
delete!(model)

true

In [8]:
# Now let's make some more complicated robots. We'll create a 
# new GeometryData from the box, but color it green this time.
green_box = GeometryData(box, RGBA(0., 1, 0, 0.5))
model = Visualizer(green_box)

Visualizer with path prefix Symbol[] using LCM LCMCore.LCM(Ptr{Void} @0x00007ff95745af00,RawFD(53),LCMCore.Subscription[LCMCore.Subscription{LCMCore.SubscriptionOptions{DrakeVisualizer.Comms.CommsT,DrakeVisualizer.#handle_msg#7{DrakeVisualizer.CoreVisualizer}}}(LCMCore.SubscriptionOptions{DrakeVisualizer.Comms.CommsT,DrakeVisualizer.#handle_msg#7{DrakeVisualizer.CoreVisualizer}}(DrakeVisualizer.Comms.CommsT,DrakeVisualizer.handle_msg),Ptr{Void} @0x00007ff95911f3f0)])

In [9]:
delete!(model)

true

# The Viewer Tree

Now that we've introduced the basics of loading and transforming geometries, it's time to introduce the tree model used by DrakeVisualizer.jl. The idea is pretty simple:

* A Visualizer contains a *tree* of geometries
* We add, draw, and delete geometries by using their *path*, which is the path from the root of the tree to that geometry
* We can set the transform for any node in the tree. Setting the transform for a node in the tree affects all of its descendant geometries. 


In [10]:
# First, we make an empty visualizer:
vis = Visualizer()

Visualizer with path prefix Symbol[] using LCM LCMCore.LCM(Ptr{Void} @0x00007ff959113420,RawFD(59),LCMCore.Subscription[LCMCore.Subscription{LCMCore.SubscriptionOptions{DrakeVisualizer.Comms.CommsT,DrakeVisualizer.#handle_msg#7{DrakeVisualizer.CoreVisualizer}}}(LCMCore.SubscriptionOptions{DrakeVisualizer.Comms.CommsT,DrakeVisualizer.#handle_msg#7{DrakeVisualizer.CoreVisualizer}}(DrakeVisualizer.Comms.CommsT,DrakeVisualizer.handle_msg),Ptr{Void} @0x00007ff959122a70)])

In [11]:
# We can access a particular path within the visualizer with indexing notation:
vis[:group1]

Visualizer with path prefix Symbol[:group1] using LCM LCMCore.LCM(Ptr{Void} @0x00007ff959113420,RawFD(59),LCMCore.Subscription[LCMCore.Subscription{LCMCore.SubscriptionOptions{DrakeVisualizer.Comms.CommsT,DrakeVisualizer.#handle_msg#7{DrakeVisualizer.CoreVisualizer}}}(LCMCore.SubscriptionOptions{DrakeVisualizer.Comms.CommsT,DrakeVisualizer.#handle_msg#7{DrakeVisualizer.CoreVisualizer}}(DrakeVisualizer.Comms.CommsT,DrakeVisualizer.handle_msg),Ptr{Void} @0x00007ff959122a70)])

In [12]:
# We load geometries using the same path notation:
green_box_vis = setgeometry!(vis[:group1][:greenbox], green_box)

Visualizer with path prefix Symbol[:group1,:greenbox] using LCM LCMCore.LCM(Ptr{Void} @0x00007ff959113420,RawFD(59),LCMCore.Subscription[LCMCore.Subscription{LCMCore.SubscriptionOptions{DrakeVisualizer.Comms.CommsT,DrakeVisualizer.#handle_msg#7{DrakeVisualizer.CoreVisualizer}}}(LCMCore.SubscriptionOptions{DrakeVisualizer.Comms.CommsT,DrakeVisualizer.#handle_msg#7{DrakeVisualizer.CoreVisualizer}}(DrakeVisualizer.Comms.CommsT,DrakeVisualizer.handle_msg),Ptr{Void} @0x00007ff959122a70)])

Check out the Scene Browser in the drake visualizer app. You should see folder:

    scene
      | remote tree viewer
          | group1
              | greenbox

  

In [13]:
# And we set transforms in the same way:
settransform!(green_box_vis, Translation(0, 0, 1))

true

In [14]:
# The same indexing notation makes it easy to get a handle to a 
# particular part of the viewer tree:
group1 = vis[:group1]
settransform!(group1, Translation(0, 0, -1))

true

The viewer tree becomes much more interesting (and useful) when we have multiple geometries. Let's add another geometry in the same group:

In [15]:
setgeometry!(vis[:group1][:bluebox], GeometryData(box, RGBA(0, 0, 1, 0.5)))

Visualizer with path prefix Symbol[:group1,:bluebox] using LCM LCMCore.LCM(Ptr{Void} @0x00007ff959113420,RawFD(59),LCMCore.Subscription[LCMCore.Subscription{LCMCore.SubscriptionOptions{DrakeVisualizer.Comms.CommsT,DrakeVisualizer.#handle_msg#7{DrakeVisualizer.CoreVisualizer}}}(LCMCore.SubscriptionOptions{DrakeVisualizer.Comms.CommsT,DrakeVisualizer.#handle_msg#7{DrakeVisualizer.CoreVisualizer}}(DrakeVisualizer.Comms.CommsT,DrakeVisualizer.handle_msg),Ptr{Void} @0x00007ff959122a70)])

The scene browser will now show:

    scene
      | remote tree viewer
          | group1
              | greenbox
              | bluebox

Since both the green and blue box are inside `group1`, we can move them together by transforming that whole group:

In [16]:
settransform!(vis[:group1], Translation(1, 0, 0))

true

Or we can move just the green box:

In [17]:
settransform!(vis[:group1][:greenbox], Translation(0, 1, 0))

true

The final pose of each geometry in the tree is just the composition of all of the transforms in the path from the root of the tree to that geometry. 

Let's try interactively moving the entire group (with the first slider) and also just the green box (with the second slider):

In [18]:
@manipulate for x1 in linspace(0, 2), x2 in linspace(0, 2)
    settransform!(vis[:group1], Translation(x1, 0, 0))
    settransform!(vis[:group1][:greenbox], Translation(x2, 1, 0))
end

Interact.Options{:SelectionSlider,Float64}(Signal{Float64}(0.9795918367346939, nactions=1),"x1",0.9795918367346939,"0.9795918367346939",Interact.OptionDict(DataStructures.OrderedDict("0.0"=>0.0,"0.04081632653061224"=>0.0408163,"0.08163265306122448"=>0.0816327,"0.12244897959183673"=>0.122449,"0.16326530612244897"=>0.163265,"0.20408163265306123"=>0.204082,"0.24489795918367346"=>0.244898,"0.2857142857142857"=>0.285714,"0.32653061224489793"=>0.326531,"0.3673469387755102"=>0.367347…),Dict(2.0=>"2.0",1.59184=>"1.5918367346938775",0.122449=>"0.12244897959183673",1.95918=>"1.9591836734693877",1.22449=>"1.2244897959183674",1.63265=>"1.6326530612244898",0.693878=>"0.6938775510204082",1.10204=>"1.1020408163265305",1.79592=>"1.7959183673469388",0.367347=>"0.3673469387755102"…)),Any[],Any[],true,"horizontal")

Interact.Options{:SelectionSlider,Float64}(Signal{Float64}(0.9795918367346939, nactions=1),"x2",0.9795918367346939,"0.9795918367346939",Interact.OptionDict(DataStructures.OrderedDict("0.0"=>0.0,"0.04081632653061224"=>0.0408163,"0.08163265306122448"=>0.0816327,"0.12244897959183673"=>0.122449,"0.16326530612244897"=>0.163265,"0.20408163265306123"=>0.204082,"0.24489795918367346"=>0.244898,"0.2857142857142857"=>0.285714,"0.32653061224489793"=>0.326531,"0.3673469387755102"=>0.367347…),Dict(2.0=>"2.0",1.59184=>"1.5918367346938775",0.122449=>"0.12244897959183673",1.95918=>"1.9591836734693877",1.22449=>"1.2244897959183674",1.63265=>"1.6326530612244898",0.693878=>"0.6938775510204082",1.10204=>"1.1020408163265305",1.79592=>"1.7959183673469388",0.367347=>"0.3673469387755102"…)),Any[],Any[],true,"horizontal")

true

Likewise, we can delete an entire group:

In [19]:
delete!(vis[:group1])

true

In [20]:
# Of course, we can draw much more interesting geometries than 
# just simple boxes. Let's load a 3D mesh and visualize it:
using MeshIO
using FileIO
cat_mesh = load(joinpath(Pkg.dir("GeometryTypes"), "test", "data", "cat.obj"))
model = Visualizer(cat_mesh);
settransform!(model, LinearMap(AngleAxis(pi/2, 1, 0, 0)))

true

In [21]:
delete!(model)

true

In [22]:
# Next, let's create a triangulated mesh by finding
# the 0-level set of some function. 
# 
# First, we'll define our function:
f = x -> sum(sin(5 * x))

# Then we pick a region of interest in which to sample the function.
# This region starts at (-1, -1, -1) and extends to (1, 1, 1):
lower_bound = Vec(-1.,-1,-1)
upper_bound = Vec(1., 1, 1)

# Those two pieces of information are all we need to construct a robot
# geometry. For this, we'll need the contour_mesh function:
mesh = contour_mesh(f, lower_bound, upper_bound)
# Under the hood, this will sample f at regularly spaced points inside
# the bounding rectangle, then compute a surface that connects all the 
# points for which f(x) = 0.

# And now we can load that geometry into the visualizer
model = Visualizer(mesh)

Visualizer with path prefix Symbol[] using LCM LCMCore.LCM(Ptr{Void} @0x00007ff9577dfe90,RawFD(71),LCMCore.Subscription[LCMCore.Subscription{LCMCore.SubscriptionOptions{DrakeVisualizer.Comms.CommsT,DrakeVisualizer.#handle_msg#7{DrakeVisualizer.CoreVisualizer}}}(LCMCore.SubscriptionOptions{DrakeVisualizer.Comms.CommsT,DrakeVisualizer.#handle_msg#7{DrakeVisualizer.CoreVisualizer}}(DrakeVisualizer.Comms.CommsT,DrakeVisualizer.handle_msg),Ptr{Void} @0x00007ff95923df70)])

In [23]:
# We can even manipulate the geometry by changing the iso level. 
# By default, contour_mesh constructs a mesh connecting the points 
# in space for which f(x) = 0, where 0 is called the isosurface level
# or iso level. But we can change that iso level to any number we want:

f = x -> sum(sin(5 * x))
lower_bound = Vec(-1.,-1,-1)
upper_bound = Vec(1., 1, 1)

@manipulate for iso_level in linspace(-1, 1, 51)
    geometry = contour_mesh(f, lower_bound, upper_bound, iso_level)
    model = Visualizer(geometry)
end

# Note that for high iso_level values our geometry gets cut off at the
# edges. We could fix that by replacing the bounds with a bigger box. 

Interact.Options{:SelectionSlider,Float64}(Signal{Float64}(0.0, nactions=1),"iso_level",0.0,"0.0",Interact.OptionDict(DataStructures.OrderedDict("-1.0"=>-1.0,"-0.96"=>-0.96,"-0.92"=>-0.92,"-0.88"=>-0.88,"-0.84"=>-0.84,"-0.8"=>-0.8,"-0.76"=>-0.76,"-0.72"=>-0.72,"-0.68"=>-0.68,"-0.64"=>-0.64…),Dict(-0.92=>"-0.92",-0.28=>"-0.28",-0.04=>"-0.04",0.56=>"0.56",-0.32=>"-0.32",0.2=>"0.2",-0.68=>"-0.68",-0.96=>"-0.96",0.48=>"0.48",-0.8=>"-0.8"…)),Any[],Any[],true,"horizontal")

Visualizer with path prefix Symbol[] using LCM LCMCore.LCM(Ptr{Void} @0x00007ff95764f360,RawFD(77),LCMCore.Subscription[LCMCore.Subscription{LCMCore.SubscriptionOptions{DrakeVisualizer.Comms.CommsT,DrakeVisualizer.#handle_msg#7{DrakeVisualizer.CoreVisualizer}}}(LCMCore.SubscriptionOptions{DrakeVisualizer.Comms.CommsT,DrakeVisualizer.#handle_msg#7{DrakeVisualizer.CoreVisualizer}}(DrakeVisualizer.Comms.CommsT,DrakeVisualizer.handle_msg),Ptr{Void} @0x00007ff955289c10)])

In [24]:
delete!(model)

true

# Batching for Better Performance

Each time you call `setgeometry!()`, `settransform!()`, or `delete!()`, there is some communication between Julia and the Drake Visualizer application. That communication can take time, so if you are loading or drawing many geometries simultaneously, then you may want to batch those operations. Batching is easy. Instead of doing this:


In [25]:
vis = Visualizer()
setgeometry!(vis[:group1][:box1], green_box)
setgeometry!(vis[:group1][:box2], green_box)
settransform!(vis[:group1], Translation(0, 1, 0))
settransform!(vis[:group1][:box1], Translation(1, 0, 0))

true

Just put all the commands inside a call to `batch()`:

In [26]:
vis = Visualizer()
batch(vis) do v
    setgeometry!(v[:group1][:box1], green_box)
    setgeometry!(v[:group1][:box2], green_box)
    settransform!(v[:group1], Translation(0, 1, 0))
    settransform!(v[:group1][:box1], Translation(1, 0, 0))
end

true

The above syntax will perform exactly the same drawing operations, but will only send one message to the viewer at the end of the `batch()` call. 

In [27]:
delete!(vis)

true

# More Geometry Types

`DrakeVisualizer` supports a few extra types of geometry that you may want to visualize. One such geometry is a point cloud: 

In [28]:
# A PointCloud can be constructed from a vector of points.
# The easiest way to represent a point is just a standard Julia vector:
pointcloud = PointCloud([[x, 0, 0] for x in linspace(-1, 1)])
setgeometry!(vis[:pointcloud], pointcloud)

# For more efficient point clouds, try using a vector of
# StaticArrays.SVector types. 

Visualizer with path prefix Symbol[:pointcloud] using LCM LCMCore.LCM(Ptr{Void} @0x00007ff959c81870,RawFD(293),LCMCore.Subscription[LCMCore.Subscription{LCMCore.SubscriptionOptions{DrakeVisualizer.Comms.CommsT,DrakeVisualizer.#handle_msg#7{DrakeVisualizer.CoreVisualizer}}}(LCMCore.SubscriptionOptions{DrakeVisualizer.Comms.CommsT,DrakeVisualizer.#handle_msg#7{DrakeVisualizer.CoreVisualizer}}(DrakeVisualizer.Comms.CommsT,DrakeVisualizer.handle_msg),Ptr{Void} @0x00007ff953705e70)])

In [29]:
using ColorTypes: RGB

In [30]:
# PointClouds can also have "channels" which describe their
# data. One useful channel is the :rgb channel:
delete!(vis[:pointcloud])
pointcloud.channels[:rgb] = [RGB(0, g, 0) for g in linspace(0, 1)]
setgeometry!(vis[:pointcloud], pointcloud)

Visualizer with path prefix Symbol[:pointcloud] using LCM LCMCore.LCM(Ptr{Void} @0x00007ff959c81870,RawFD(293),LCMCore.Subscription[LCMCore.Subscription{LCMCore.SubscriptionOptions{DrakeVisualizer.Comms.CommsT,DrakeVisualizer.#handle_msg#7{DrakeVisualizer.CoreVisualizer}}}(LCMCore.SubscriptionOptions{DrakeVisualizer.Comms.CommsT,DrakeVisualizer.#handle_msg#7{DrakeVisualizer.CoreVisualizer}}(DrakeVisualizer.Comms.CommsT,DrakeVisualizer.handle_msg),Ptr{Void} @0x00007ff953705e70)])

In [31]:
delete!(vis)

true

In [27]:
# Close the viewer:
# kill(proc)